#Notebook

This notebook comprises the epxorting of google earth engine images to tiff files/images which are ready to be processed to tfrecords as training images

Here images are filtered by
 -  time
 - region
 - fit into tiles
 - exported/dowloaded to local google drive as tiff images

# Install libraries

In [ ]:

!pip -q install -U geemap
!pip -q install geopandas shapely



     |████████████████████████████████| 476 kB 5.3 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 76 kB 5.2 MB/s 
     |████████████████████████████████| 219 kB 46.9 MB/s 
     |████████████████████████████████| 1.3 MB 51.1 MB/s 
     |████████████████████████████████| 1.6 MB 23.7 MB/s 
     |████████████████████████████████| 3.3 MB 18.9 MB/s 
     |████████████████████████████████| 130 kB 69.4 MB/s 
     |████████████████████████████████| 98 kB 7.0 MB/s 
     |████████████████████████████████| 93 kB 1.3 MB/s 
     |████████████████████████████████| 1.2 MB 52.1 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 70 kB 7.3 MB/s 
     |████████████████████████████████| 393 kB 62.7 MB/s 
     |████████████████████████████████| 553 kB 61.2 MB/s 
     |████████████████████████████████| 112 kB 49.8 MB/s 
     |████████████████████████████████| 428 kB 49.2 MB/s 
     |█████████████████████

In [ ]:
import geopandas as gpd
import glob
import pandas as pd
import os

# import geetools
import tensorflow as tf

### Rasterio

In [ ]:
!pip install -q rasterio

     |████████████████████████████████| 19.3 MB 56.3 MB/s 


In [ ]:
import rasterio

#Authenticate


## Authenticate to GEE


In [ ]:

import ee
import geemap.eefolium as geemap
ee.Authenticate()
ee.Initialize()




To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=9r3YXuF4QfZY4aP4d4jkMktrpW_z5_bw7WBqy1ZsW40&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgoDrTyjsz6XKkqzpfJNlhSi7I8na3njQbLfSslSfShuzX02HVrtaE

Successfully saved authorization token.


## Authenticate to Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pathExp = "dB2709/"

# Import data

##Load assets: y data and tiles
The y data are the masked target images. The NonLS images are black, the LS images are black and white. The landslide polygons are colored white.
The tiles are geometries, tiles produced by rasterization, corresponding to approx. 2 by 2 km^2. The satellite images will be exported with the size of these tiles. 


In [ ]:
tilesHiro = ee.FeatureCollection("users/marjamachielsegg/HiroTiles")
tilesHokkaido = ee.FeatureCollection("users/marjamachielsegg/HokTiles")
tilesHu = ee.FeatureCollection("users/marjamachielsegg/HuTiles")
tilesLombok = ee.FeatureCollection("users/marjamachielsegg/LombokTiles")


In [ ]:
HiroY = ee.Image("users/marjamachielsegg/HiroY10")
HuY = ee.Image("users/marjamachielsegg/HuY10m")
HokY = ee.Image("users/marjamachielsegg/HokY10m_2")
LombokY = ee.Image("users/marjamachielsegg/LombokY10m")

In [ ]:
HiroYNonLS = ee.Image("users/marjamachielsegg/HiroYNonLS20m")
HuYNonLS = ee.Image("users/marjamachielsegg/HuYNonLS20m_2")
HokYNonLS = ee.Image("users/marjamachielsegg/HokYNonLS20m")
LombokYNonLS = ee.Image("users/marjamachielsegg/LombokYNonLS20m")

## Load ImageCollections from Google Drive and make LS inventory dictionary

In [ ]:
# L8TOA = ee.ImageCollection("LANDSAT/LC08/C01/T1_RT_TOA")
#use dB here!
S1dB = ee.ImageCollection("COPERNICUS/S1_GRD")
# S2 = ee.ImageCollection("COPERNICUS/S2")

#Might need to redo these geometries, to make them slightly bigger so that they include all polygons
Hiroshima = ee.Geometry.Polygon(
    [[[132.6862647368235, 34.15717773816654],
      [132.9389502836985, 34.275280475796855],
      [133.230087979011, 34.32973356797482],
      [133.559677822761, 34.479297379909966],
      [133.5761573149485, 34.61503224648198],
      [133.186142666511, 34.700883075516785],
      [133.021347744636, 34.60599014490323],
      [132.3676612211985, 34.51098864780678],
      [132.329209072761, 34.261661687255],
      [132.4335791899485, 34.15717773816654]]])
Hokkaido = ee.Geometry.Polygon(
    [[[141.88807524167268, 42.94444603183977],
      [141.78919828854768, 42.85189180928013],
      [141.80567778073518, 42.69059710692225],
      [141.95948637448518, 42.56530485503135],
      [142.15527119607927, 42.58957464929461],
      [142.16625752420427, 42.78742433456754],
      [142.08935322732927, 42.867998174992415]]])
China = ee.Geometry.Polygon(
    [[[95.0742421678175, 29.999903555631697],
      [94.86275535141125, 29.74029674997442],
      [95.18959861313, 29.62576058848347],
      [95.38784688756272, 29.828495879457826]]])
Lombok = ee.Geometry.Polygon(
    [[[116.30332651211572, -8.568466262153649],
      [116.62604990078759, -8.462530449782491],
      [116.66724863125634, -8.309007726713741],
      [116.48322763516259, -8.242417181981912],
      [116.23328867031884, -8.270957365010824],
      [116.04789438320947, -8.4543803292391],
      [116.10831918789697, -8.527725191173909]]])
# visParams = {"bands":["swir","nir","red"],"min":0.05,"max":0.4}
L8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
# S1Float = ee.ImageCollection("COPERNICUS/S1_GRD_FLOAT")
# ColorScale = {"min":-3,"max":3,"palette":["ffffff","ffffff","ff0000"]}

#Take slope instead of elevation
srtm = ee.Image("USGS/SRTMGL1_003")
slope = ee.Terrain.slope(srtm.select("elevation"))

In [ ]:
dbLS = [
  {
    'name': "Hiroshima",
    # 'ft': eeHiroshima,
    'geom': Hiroshima,
    'tiles': tilesHiro,
    'eventDate' : "2018-07-07", #by Adriano; most happend evening 6, morning 7
    't1': "2018-06-28",
    't2': "2018-07-09",
    'amountOfTiles':1088,
    'YLS':HiroY,
    'YNonLS':HiroYNonLS
  },
  {
    'name': "Hu",
    # 'ft': eeHu,
    'geom': China,
    'tiles': tilesHu,
    'eventDate' : "2017-11-17", #or 18..
    't1': "2017-11-16",
    't2':"2017-11-19",
    'amountOfTiles':292,
    'YLS':HuY,
    'YNonLS':HuYNonLS
  },
  {
    'name': "FerrarioMerge", 
    # 'ft': eeLombokMerge, 
    'tiles': tilesLombok,
    'geom': Lombok,
    'eventDate' : "2018-07-29", #for consec used FerB, with eventDate 2018-08-05..
    't1':"2018-07-28",
    't2': "2018-08-20",
    'amountOfTiles':331,
    'YLS':LombokY,
    'YNonLS':LombokYNonLS
  },
  {
    'name': "HokkaidoMerge",
    # 'ft':eeHokkaidoMerge, 
    'geom': Hokkaido,
    'tiles': tilesHokkaido,
    'eventDate' : "2018-10-06", #why not 2018-10-06
    't1': "2018-10-06",
    't2':"2018-10-07",#last aftershock 21 feb 2019?
    'amountOfTiles':336,
    'YLS':HokY,
    'YNonLS':HokYNonLS
  }
  ]


## Export DEM data

In [1]:
def exportOneImage_dem(index,image,image_projection, tile_nr, folder_name):
  """
  This function exports one image from the Digital Elevation Model (DEM) satellite imagery.
  This function is called by the function export_dem, which calls all the images for one region/database.
  The input variable 'index' refers to the index of the landslide database, which region/database of the four databases is selected.
  These four databased are summed in the dbLS. 
  [0] refers to Hiroshima, Japan
  [1] refers to Tsangpo Gorge, Chinna
  [2] refers to Lombok, Indonesia
  [3] refers to Hokkaido, Japan

  The input variable 'image' refers to the image which is to be exported

  The input variable 'image_projection' refers to the projection of the image which is to be exported

  The 'region' variable depicts the region which the image represents, here this is bound to a certain tile. 
  Which tile this is, from the amount of tiles available for each region, is defined by the input variable 'tile_nr'

  The input varialbe 'folder_name' refers to the name of the folder in the Drive to which the exported images
  are to be saved. This folder is made by the function but can also be made beforehand.

  The image is of dimension 256 by 256 pixels, this can be changed at variable 'dimensions'
  The name of the image is defined by 'name_image', currently it is chosen to change that automatically
  wrt the tile number.
  """
  tiles = dbLS[index]["tiles"]
  tiles_amount = tiles.size().getInfo()
  tiles_list = tiles.toList(tiles_amount)
  geom_tile = ee.Feature(tiles_list.get(tile_nr)).geometry().bounds(1,image_projection)
  name_image = "ImageDEM_%s" % (tile_nr) #ImageIMAGENUMBERFROMCOLL_TILENUMBER'
  ee.batch.Export.image.toDrive(
  image = image,
  dimensions = "256x256",          
  region = geom_tile,
  description = name_image,
  fileNamePrefix = name_image, #necessary next to descirptioN?
  folder = folder_name).start()

In [ ]:
def export_dem(index,folder_name_dem):
  """
  This function exports all images of the Digital Elevation Model (DEM) satellite imagery for a certain landslide database/area. 
  The input variable 'index' refers to the index of the landslide database, which region/database of the four databases is selected.
  These four databased are summed in the dbLS. 
  [0] refers to Hiroshima, Japan
  [1] refers to Tsangpo Gorge, Chinna
  [2] refers to Lombok, Indonesia
  [3] refers to Hokkaido, Japan

  The input varialbe 'folder_name_dem' refers to the name of the folder in the Drive to which the exported images
  are to be saved. This folder is made by the function but can also be made beforehand.
  """
  # // LOAD Shuttle Radar Topography Mission (SRTM) Digital Elevation Model (DEM)
  dataset = ee.Image('USGS/SRTMGL1_003')
  elevation = dataset.select('elevation') 
  slope = ee.Terrain.slope(elevation);
  #get tiles
  tiles = dbLS[index]["tiles"]
  tiles_amount = tiles.size().getInfo()
  
  image_projection = slope.projection()
  for i in range(tiles_amount):
    tile_nr = i + 1
    exportOneImage_dem(index,slope,image_projection,tile_nr,folder_name_dem)



## Export Y data

In [ ]:
S1 = S1dB

In [ ]:
def getImageProjection(index):
  t1 = dbLS[index]["t1"]
  t2 = dbLS[index]["t2"]
  images = S1.filterDate(t1,t2).filterBounds(dbLS[index]["geom"]).filter(ee.Filter.eq('resolution_meters', 10))
  imagesVH =  images.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).select('VH')
  imagesdes = imagesVH.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).sort('system:time_start')
  image = imagesdes.first()
  imageProjection = image.select(0).projection()
  return imageProjection

In [ ]:
def exportImageYLS(folderNameLS,index):
  """
  This function exports all images of the target images which contain landslide polygons for a certain landslide database/area. 
  The input variable 'index' refers to the index of the landslide database, which region/database of the four databases is selected.
  These four databased are summed in the dbLS. 
  [0] refers to Hiroshima, Japan
  [1] refers to Tsangpo Gorge, Chinna
  [2] refers to Lombok, Indonesia
  [3] refers to Hokkaido, Japan

  The input varialbe 'folder_nameLS' refers to the name of the folder in the Drive to which the exported images
  are to be saved. This folder is made by the function but can also be made beforehand.


  The image is of dimension 256 by 256 pixels, this can be changed at variable 'dimensions'
  The name of the image is defined by 'name_image', currently it is chosen to change that automatically
  wrt the tile number.
  """
  imageYLS = dbLS[index]["YLS"]
  tilesToUse = dbLS[index]["tiles"]
  amountOfTiles = tilesToUse.size().getInfo()
  print(amountOfTiles)
  tilesList = tilesToUse.toList(amountOfTiles)


  imageProjection = getImageProjection(index) #sar band, proj of des
  for i in range(amountOfTiles): 
    geomTile = ee.Feature(tilesList.get(i)).geometry().bounds(1,imageProjection)
    nameImageLS = "ImageLS_%s" % (i) #Image_TILENUMBER
    ee.batch.Export.image.toDrive(
    image = imageYLS,
    dimensions = "256x256",           
    region = geomTile,
    description = nameImageLS,
    fileNamePrefix = nameImageLS,
    folder = folderNameLS).start()



#Functions

### Functions to get images with loaded tiles

In [ ]:

# def getImagesVH(ic):
#   imgVH = ic.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).select('VH')

#   #do some kind of DEM filtering here? or do DEM as extra band such that cGAN has two bands..
#   return imgVH

In [ ]:
## Here the slope and curvature mask is made, for details see Handwerger, A., Jones, S., Huang, M.-H., Amatya, P., Kerner, H., and
# Kirschbaum, D. (2020). Rapid landslide identification using synthetic aperture
# radar amplitude change detection on the Google Earth Engine. Nat. Hazards
# Earth Syst. Sci.

slope_threshold = .5#; // unit: degree
curv_threshold = -0.005#; // unit: m/m^2

# // LOAD Shuttle Radar Topography Mission (SRTM) Digital Elevation Model (DEM)
dataset = ee.Image('USGS/SRTMGL1_003')
elevation = dataset.select('elevation') 
slope = ee.Terrain.slope(elevation) #//slope in degrees
mask_slope = slope.gte(slope_threshold) #// slope mask with values 0 or 1
slope_mask = slope.updateMask(mask_slope)


In [ ]:

# // Calculate curvature
# // Define a Gaussian kernel for smoothing. This step helps reduce noise in the curvature maps
# smooth_curv = ee.Kernel.gaussian({radius = 120,sigma = 60,units = 'meters',normalize = true})

smooth_curv = ee.Kernel.gaussian(radius = 120,sigma = 60,units = 'meters',normalize = True)
xyDemGrad = elevation.convolve(smooth_curv).gradient()
xGradient = xyDemGrad.select('x').gradient()
yGradient = xyDemGrad.select('y').gradient()
curvature = xGradient.select('x').add(yGradient.select('y'))
mask_curvature = curvature.gte(curv_threshold)
curvature_mask = slope.updateMask(mask_curvature)

# // Define a Gaussian kernel to reduce noise in S1 scenes
smooth_S1 = ee.Kernel.gaussian(radius = 50,sigma = 20, units = 'meters',normalize = True)


In [ ]:
def neg30(image):
  """"
  This function removes all pixels which have a value which is smaller than -30 dB from an image,
  defined by the input variable "image", whic is recommended for the Copernicus Sentinel-1 imagery from Google Earth Engine.
  Returned is the image, with said pixels removed
  """"
  return image.updateMask(image.gt(-30.0))


def getImagesVH(ic):
  """"
  This function filter the input variable "ic", which is an image collection of Copernicus Sentinel-1 imagery from Google Earth Engine,
  on VH polarization, IW mode and implements the neg30 function. Returned is an image collection, which is filtered on the aforementioned.
  """"
  imgVH = ic.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW')).select('VH')
  imgVH_no_neg30 = imgVH.map(neg30) #Then these pixels become 0
  return imgVH_no_neg30



In [ ]:
def getDes(ic):
  """"
  This function returns the input variable "ic", which is an image collection of Copernicus Sentinel-1 imagery from Google Earth Engine,
  with only images of orbit with descending direction.
  """"
  des = ic.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).sort('system:time_start')
  return des


In [ ]:
def getAsc(ic):
  """"
  This function returns the input variable "ic", which is an image collection of Copernicus Sentinel-1 imagery from Google Earth Engine,
  with only images of orbit with ascending direction.
  """"
  asc = ic.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).sort('system:time_start')
  return asc

In [ ]:
def getImagesPre(S,index, dateRange,stringMWY):
  """"
  This function returns the pre-event images, which is to say images which have happened before the landslide event happens. 
  The input variable "S" refers to the image collection of Copernicus Sentinel-1 imagery from Google Earth Engine, the version which is in decibel.
  The input variable 'index' refers to the index of the landslide database, which region/database of the four databases is selected.
  These four databased are summed in the dbLS. 
  [0] refers to Hiroshima, Japan
  [1] refers to Tsangpo Gorge, Chinna
  [2] refers to Lombok, Indonesia
  [3] refers to Hokkaido, Japan

  The input variable 'dateRange' refers to the amount of time, and is given in integers.
  The input variable 'stringMWY' refers to the unit of time, is given in a string, and can be either :
  'day','month','year'.
  Together they make up the amount of time during which the images can be selected before the event date, e.g. 
  'dateRange' = 1 , stringMWY = 'month' gives images during the timespan of 1 month before the event date.
  The event date can be found in the dbLS. The pre-event date and post-event date can differ per event, this is explained in the thesis. 

  Images of orbit with ascending and descending direction are returned separetely.
  """"
  db = dbLS[index]
  # t1 = ee.Date(db["eventDate"]).advance(-dateRange,'month')
  # t2 = ee.Date(db["eventDate"]).advance(dateRange,'month')
  t1 = dbLS[index]["t1"]
  t1Pre = ee.Date(t1).advance(-dateRange,stringMWY)

  #filter on main geom, which contains all Polygons
  imagesPre = S.filterDate(t1Pre,t1).filterBounds(dbLS[index]["geom"]).filter(ee.Filter.eq('resolution_meters', 10))
 

  imagesPreVH = getImagesVH(imagesPre)
  desPre = getDes(imagesPreVH)
  ascPre = getAsc(imagesPreVH)
  return ascPre,desPre


In [ ]:
def getImagesPost(S,index, dateRange,stringMWY):
  """"
  This function returns the post-event images, which is to say images which have happened after the landslide event happens. 
  The input variable "S" refers to the image collection of Copernicus Sentinel-1 imagery from Google Earth Engine, the version which is in decibel.
  The input variable 'index' refers to the index of the landslide database, which region/database of the four databases is selected.
  These four databased are summed in the dbLS. 
  [0] refers to Hiroshima, Japan
  [1] refers to Tsangpo Gorge, Chinna
  [2] refers to Lombok, Indonesia
  [3] refers to Hokkaido, Japan

  The input variable 'dateRange' refers to the amount of time, and is given in integers.
  The input variable 'stringMWY' refers to the unit of time, is given in a string, and can be either :
  'day','month','year'.
  Together they make up the amount of time during which the images can be selected after the event date, e.g. 
  'dateRange' = 1 , stringMWY = 'month' gives images during the timespan of 1 month after the event date.
  The event date can be found in the dbLS. The pre-event date and post-event date can differ per event, this is explained in the thesis. 

  Images of orbit with ascending and descending direction are returned separetely.
  """"
  
  db = dbLS[index]
  t2 = dbLS[index]["t2"]
  t2Post = ee.Date(t2).advance(dateRange,stringMWY)

  
  imagesPost = S.filterDate(t2,t2Post).filterBounds(dbLS[index]["geom"]).filter(ee.Filter.eq('resolution_meters', 10))
  imagesPostVH = getImagesVH(imagesPost)
  desPost = getDes(imagesPostVH)
  ascPost = getAsc(imagesPostVH)
  # return ascPost
  return ascPost,desPost


### Function to get which image is in tile

In [ ]:
def addMaskedAreaRatio(image,tile):
  """"
  The function returns the ratio of image in tile. It refers to the amount of tile which is covered by the image.
  This is necessary since some images only cover the tile for 10% or 60%,
   and the rest of the exported image bound bu the geometry of the tile will contain nan values.
  The input variable 'image' defines the image, The input variable 'tile' defines the tile.   
  """"
  areaNonEmpty = image.pixelArea().updateMask(image.mask()).reduceRegion(ee.Reducer.sum(),tile.geometry(),100).values().get(0)
  area = tile.geometry().area(1)
  ratioNonEmpty = ee.Number(areaNonEmpty).divide(area)
  #could also make ratioNonEmpty a property of the image
  # return image.set(ratioNonEmpty:rationNonEmpty)
  return ratioNonEmpty

    


### Projection image

In [ ]:
def getProjection_eeList(imageList):
  """"
  This function returns the image projection from a ee.List of image as input
  """"
  for i in range(len(imageList)):
    if (imageList[i]): #if non empty
      imageProjection = ee.Image(imageList[i].get(0)).select(0).projection()
      break
  return imageProjection

In [ ]:
def getProjection_List(imageList):
  """"
  This function returns the image projection from a pythonic list of image as input
  """"
  for i in range(len(imageList)):
    if (imageList[i]): #if non empty
      imageProjection = ee.Image(imageList[i]).select(0).projection()
      break
  return imageProjection

### getImage Per Tile

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# def add_masks_to_image(image):
#   #this is done on image after taking the median
#   #convolve makes it smoother, although this wasn't used yet in Handwerger
#   #the slope and curvature mask account for slope and curvate wrt to landslides.
  
#   #with smoothing
#   img_masked = image.convolve(smooth_S1).updateMask(mask_slope).updateMask(mask_curvature) #has to go over image instead of imcoll
  
#   #without smoothing
#   # img_masked = image.updateMask(mask_slope).updateMask(mask_curvature)

#   return img_masked

In [ ]:
def get_image_per_tile(S,index, date_range_num_pre,date_range_num_post,string_w_m_y,ratio_criteria):
  """"
  This function returns the A ratio images and their respective tile number, in a double list form.
  The A ratio is the log-based amplitude difference, which is defined by A ratio = A pre - A post.
  This is done for both images of descending and ascending direction, which are averaged together.


  The input variable "S" refers to the image collection of Copernicus Sentinel-1 imagery from Google Earth Engine, the version which is in decibel.
  The input variable 'index' refers to the index of the landslide database, which region/database of the four databases is selected.
  These four databased are summed in the dbLS. 
  [0] refers to Hiroshima, Japan
  [1] refers to Tsangpo Gorge, Chinna
  [2] refers to Lombok, Indonesia
  [3] refers to Hokkaido, Japan

  The input variable 'date_range_num_pre' refers to the amount of time before the event, and is given in integers.
  The input variable 'date_range_num_post' refers to the amount of time after the event, and is given in integers
  The input variable 'string_w_m_y' refers to the unit of time, is given in a string, and can be either :
  'day','week','month','year'.

  Together they make up the amount of time during which the images can be selected after the event date, e.g. 
  'date_range_num_pre' = 1 , string_w_m_y = 'month' gives images during the timespan of 1 month after the event date.
  The event date can be found in the dbLS. The pre-event date and post-event date can differ per event, this is explained in the thesis. 

  The input variable "ratio_criteria" gives the threshold ratio of which the images has to cover the tile. Recommended is to put ratio_criteria to 0.9.

  The addition of slope and curvature masks can be done by uncommenting the '#add masks' lines,
  and commention the "#without masks" lines. Vice versa for no addition of slope and curvature masks. 
  """"

  #get pre event images and their amounts. Turn to list, necessary for func
  images_pre_asc,images_pre_des = getImagesPre(S,index, date_range_num_pre,string_w_m_y)
  #asc
  amount_pre_asc = images_pre_asc.size().getInfo()
  im_pre_asc_list = images_pre_asc.toList(amount_pre_asc)
  #des
  amount_pre_des = images_pre_des.size().getInfo()
  im_pre_des_list = images_pre_des.toList(amount_pre_des)

  #get post event images and their amounts. Turn to list, necessary for func
  images_post_asc,images_post_des = getImagesPost(S,index, date_range_num_pre,string_w_m_y)
  #asc
  amount_post_asc = images_post_asc.size().getInfo()
  im_post_asc_list = images_post_asc.toList(amount_post_asc)
  #des
  amount_post_des = images_post_asc.size().getInfo()
  im_post_des_list = images_post_des.toList(amount_post_des)

  #tiles
  tiles = dbLS[index]["tiles"]
  tiles_amount = tiles.size().getInfo()
  tiles_list = tiles.toList(tiles_amount)

  #As output, both pre and post sep.
  # sum_pre_out = []
  # sum_post_out = []
  sum_out = [[],[]]


#Get output with double for loop, over tiles over images (pre and post sep.)
  for i in range(tiles_amount):
  # for i in range(2):
    # tile_nr = i + 1
    pre_tile_asc = []
    post_tile_asc = []
    pre_tile_des = []
    post_tile_des = []
 

    tile = ee.Feature(tiles_list.get(i))

    #get al images per tile for pre asc
    for j in range(amount_pre_asc):
      image = ee.Image(im_pre_asc_list.get(j))
      ratio_temp = addMaskedAreaRatio(image,tile) #could be faster it tile area is same everyhwere, to calc. that outside 2nd loop
      ratio = ratio_temp.getInfo()
      if (ratio > ratio_criteria):
        pre_tile_asc.append(image)
        #stack the im immediately in pre_tile_asc ? and then + des ... --> then possible to do (asc+des)/2


    #get al images per tile for pre des
    for k in range(amount_pre_des):
      image = ee.Image(im_pre_des_list.get(k))
      ratio_temp = addMaskedAreaRatio(image,tile) #could be faster it tile area is same everyhwere, to calc. that outside 2nd loop
      ratio = ratio_temp.getInfo()
      if (ratio > ratio_criteria):
        pre_tile_des.append(image)

 
#get al images per tile for pre asc
    for l in range(amount_post_asc):
      image = ee.Image(im_post_asc_list.get(l))
      ratio_temp = addMaskedAreaRatio(image,tile) #could be faster it tile area is same everyhwere, to calc. that outside 2nd loop
      ratio = ratio_temp.getInfo()
      if (ratio > ratio_criteria):
        post_tile_asc.append(image)
      

    #get al images per tile for pre des
    for m in range(amount_post_des):
      image = ee.Image(im_post_des_list.get(m))
      ratio_temp = addMaskedAreaRatio(image,tile) #could be faster it tile area is same everyhwere, to calc. that outside 2nd loop
      ratio = ratio_temp.getInfo()
      if (ratio > ratio_criteria):
        post_tile_des.append(image)
      
    #possbily multiple images per tile --> need to stack and average those per tile and 

    im_coll_pre_tile_asc = ee.ImageCollection(pre_tile_asc) 
    im_coll_pre_tile_des = ee.ImageCollection(pre_tile_des)
    im_coll_post_tile_asc = ee.ImageCollection(post_tile_asc) 
    im_coll_post_tile_des = ee.ImageCollection(post_tile_des) 


    #add masks.
    # pre_tile_asc_med = add_masks_to_image(im_coll_pre_tile_asc.median())
    # pre_tile_des_med =  add_masks_to_image(im_coll_pre_tile_des.median())
    # post_tile_asc_med = add_masks_to_image(im_coll_post_tile_asc.median())
    # post_tile_des_med = add_masks_to_image(im_coll_post_tile_des.median())

    #without masks
    pre_tile_asc_med = im_coll_pre_tile_asc.median()
    pre_tile_des_med =  im_coll_pre_tile_des.median()
    post_tile_asc_med = im_coll_post_tile_asc.median()
    post_tile_des_med = im_coll_post_tile_des.median()
    


    if (len(pre_tile_asc) != 0 and len(pre_tile_des) != 0 and len(post_tile_asc) != 0 and len(post_tile_des) != 0):
      diff_des = pre_tile_des_med.subtract(post_tile_des_med)
      diff_asc = pre_tile_asc_med.subtract(post_tile_asc_med)
      sum_des_asc = (diff_asc.add(diff_des)).divide(2)
      sum_out[0].append(i+1)
      sum_out[1].append(sum_des_asc)

  
  return sum_out

In [ ]:
# def export_image_per_tile(S,index, date_range_num_pre,date_range_num_post,string_w_m_y,ratio_criteria,folder_name_x):

#   #get pre event images and their amounts. Turn to list, necessary for func
#   images_pre_asc,images_pre_des = getImagesPre(S,index, date_range_num_pre,string_w_m_y)
#   #asc
#   amount_pre_asc = images_pre_asc.size().getInfo()
#   im_pre_asc_list = images_pre_asc.toList(amount_pre_asc)
#   #des
#   amount_pre_des = images_pre_des.size().getInfo()
#   im_pre_des_list = images_pre_des.toList(amount_pre_des)

#   #get post event images and their amounts. Turn to list, necessary for func
#   images_post_asc,images_post_des = getImagesPost(S,index, date_range_num_pre,string_w_m_y)
#   #asc
#   amount_post_asc = images_post_asc.size().getInfo()
#   im_post_asc_list = images_post_asc.toList(amount_post_asc)
#   #des
#   amount_post_des = images_post_asc.size().getInfo()
#   im_post_des_list = images_post_des.toList(amount_post_des)

#   #tiles
#   tiles = dbLS[index]["tiles"]
#   tiles_amount = tiles.size().getInfo()
#   tiles_list = tiles.toList(tiles_amount)

#   #As output, both pre and post sep.
#   # sum_pre_out = []
#   # sum_post_out = []
#   # sum_out = [[],[]]


# #Get output with double for loop, over tiles over images (pre and post sep.)
#   for i in range(tiles_amount):
#   # for i in range(2):
#     # tile_nr = i + 1
#     pre_tile_asc = []
#     post_tile_asc = []
#     pre_tile_des = []
#     post_tile_des = []
 

#     tile = ee.Feature(tiles_list.get(i))

#     #get al images per tile for pre asc
#     for j in range(amount_pre_asc):
#       image = ee.Image(im_pre_asc_list.get(j))
#       ratio_temp = addMaskedAreaRatio(image,tile) #could be faster it tile area is same everyhwere, to calc. that outside 2nd loop
#       ratio = ratio_temp.getInfo()
#       if (ratio > ratio_criteria):
#         pre_tile_asc.append(image)
#         #stack the im immediately in pre_tile_asc ? and then + des ... --> then possible to do (asc+des)/2


#     #get al images per tile for pre des
#     for k in range(amount_pre_des):
#       image = ee.Image(im_pre_des_list.get(k))
#       ratio_temp = addMaskedAreaRatio(image,tile) #could be faster it tile area is same everyhwere, to calc. that outside 2nd loop
#       ratio = ratio_temp.getInfo()
#       if (ratio > ratio_criteria):
#         pre_tile_des.append(image)

 
# #get al images per tile for pre asc
#     for l in range(amount_post_asc):
#       image = ee.Image(im_post_asc_list.get(l))
#       ratio_temp = addMaskedAreaRatio(image,tile) #could be faster it tile area is same everyhwere, to calc. that outside 2nd loop
#       ratio = ratio_temp.getInfo()
#       if (ratio > ratio_criteria):
#         post_tile_asc.append(image)
      

#     #get al images per tile for pre des
#     for m in range(amount_post_des):
#       image = ee.Image(im_post_des_list.get(m))
#       ratio_temp = addMaskedAreaRatio(image,tile) #could be faster it tile area is same everyhwere, to calc. that outside 2nd loop
#       ratio = ratio_temp.getInfo()
#       if (ratio > ratio_criteria):
#         post_tile_des.append(image)
      
#     #possbily multiple images per tile --> need to stack and average those per tile and 

#     im_coll_pre_tile_asc = ee.ImageCollection(pre_tile_asc) 
#     im_coll_pre_tile_des = ee.ImageCollection(pre_tile_des)
#     im_coll_post_tile_asc = ee.ImageCollection(post_tile_asc) 
#     im_coll_post_tile_des = ee.ImageCollection(post_tile_des) 


#     # add masks.
#     pre_tile_asc_med = add_masks_to_image(im_coll_pre_tile_asc.median())
#     pre_tile_des_med =  add_masks_to_image(im_coll_pre_tile_des.median())
#     post_tile_asc_med = add_masks_to_image(im_coll_post_tile_asc.median())
#     post_tile_des_med = add_masks_to_image(im_coll_post_tile_des.median())
    


#     if (len(pre_tile_asc) != 0 and len(pre_tile_des) != 0 and len(post_tile_asc) != 0 and len(post_tile_des) != 0):
#       diff_des = pre_tile_des_med.subtract(post_tile_des_med)
#       diff_asc = pre_tile_asc_med.subtract(post_tile_asc_med)
#       sum_des_asc = (diff_asc.add(diff_des)).divide(2)
#       # sum_out[0].append(i+1)
#       # sum_out[1].append(sum_des_asc)
#       exportOneImage_LS(index,sum_des_asc,i+1, folder_name_x)

#       #maybe just immediately export this image... with proper tile..
  


In [ ]:
def export_image_per_tile(S,index, date_range_num_pre,date_range_num_post,string_w_m_y,ratio_criteria,folder_name_x):
  """"
  This function exports the A ratio images.
  The A ratio is the log-based amplitude difference, which is defined by A ratio = A pre - A post.
  This is done for both images of descending and ascending direction, which are averaged together.


  The input variable "S" refers to the image collection of Copernicus Sentinel-1 imagery from Google Earth Engine, the version which is in decibel.
  The input variable 'index' refers to the index of the landslide database, which region/database of the four databases is selected.
  These four databased are summed in the dbLS. 
  [0] refers to Hiroshima, Japan
  [1] refers to Tsangpo Gorge, Chinna
  [2] refers to Lombok, Indonesia
  [3] refers to Hokkaido, Japan

  The input variable 'date_range_num_pre' refers to the amount of time before the event, and is given in integers.
  The input variable 'date_range_num_post' refers to the amount of time after the event, and is given in integers
  The input variable 'string_w_m_y' refers to the unit of time, is given in a string, and can be either :
  'day','week','month','year'.

  Together they make up the amount of time during which the images can be selected after the event date, e.g. 
  'date_range_num_pre' = 1 , string_w_m_y = 'month' gives images during the timespan of 1 month after the event date.
  The event date can be found in the dbLS. The pre-event date and post-event date can differ per event, this is explained in the thesis. 

  The input variable "ratio_criteria" gives the threshold ratio of which the images has to cover the tile. Recommended is to put ratio_criteria to 0.9.

  The addition of slope and curvature masks can be done by uncommenting the '#add masks' lines,
  and commention the "#without masks" lines. Vice versa for no addition of slope and curvature masks. 
  """"
  #get pre event images and their amounts. Turn to list, necessary for func
  images_pre_asc,images_pre_des = getImagesPre(S,index, date_range_num_pre,string_w_m_y)
  #asc
  amount_pre_asc = images_pre_asc.size().getInfo()
  im_pre_asc_list = images_pre_asc.toList(amount_pre_asc)
  #des
  amount_pre_des = images_pre_des.size().getInfo()
  im_pre_des_list = images_pre_des.toList(amount_pre_des)

  #get post event images and their amounts. Turn to list, necessary for func
  images_post_asc,images_post_des = getImagesPost(S,index, date_range_num_pre,string_w_m_y)
  #asc
  amount_post_asc = images_post_asc.size().getInfo()
  im_post_asc_list = images_post_asc.toList(amount_post_asc)
  #des
  amount_post_des = images_post_asc.size().getInfo()
  im_post_des_list = images_post_des.toList(amount_post_des)

  #tiles
  tiles = dbLS[index]["tiles"]
  tiles_amount = tiles.size().getInfo()
  tiles_list = tiles.toList(tiles_amount)

  #As output, both pre and post sep.
  # sum_pre_out = []
  # sum_post_out = []
  # sum_out = [[],[]]


#Get output with double for loop, over tiles over images (pre and post sep.)
  for i in range(tiles_amount):
  # for i in range(2):
    # tile_nr = i + 1
    pre_tile_asc = []
    post_tile_asc = []
    pre_tile_des = []
    post_tile_des = []
 

    tile = ee.Feature(tiles_list.get(i))

    #get al images per tile for pre asc
    for j in range(amount_pre_asc):
      image = ee.Image(im_pre_asc_list.get(j))
      ratio_temp = addMaskedAreaRatio(image,tile) #could be faster it tile area is same everyhwere, to calc. that outside 2nd loop
      ratio = ratio_temp.getInfo()
      if (ratio > ratio_criteria):
        pre_tile_asc.append(image)
        #stack the im immediately in pre_tile_asc ? and then + des ... --> then possible to do (asc+des)/2


    #get al images per tile for pre des
    for k in range(amount_pre_des):
      image = ee.Image(im_pre_des_list.get(k))
      ratio_temp = addMaskedAreaRatio(image,tile) #could be faster it tile area is same everyhwere, to calc. that outside 2nd loop
      ratio = ratio_temp.getInfo()
      if (ratio > ratio_criteria):
        pre_tile_des.append(image)

 
#get al images per tile for pre asc
    for l in range(amount_post_asc):
      image = ee.Image(im_post_asc_list.get(l))
      ratio_temp = addMaskedAreaRatio(image,tile) #could be faster it tile area is same everyhwere, to calc. that outside 2nd loop
      ratio = ratio_temp.getInfo()
      if (ratio > ratio_criteria):
        post_tile_asc.append(image)
      

    #get al images per tile for pre des
    for m in range(amount_post_des):
      image = ee.Image(im_post_des_list.get(m))
      ratio_temp = addMaskedAreaRatio(image,tile) #could be faster it tile area is same everyhwere, to calc. that outside 2nd loop
      ratio = ratio_temp.getInfo()
      if (ratio > ratio_criteria):
        post_tile_des.append(image)
      
    #possbily multiple images per tile --> need to stack and average those per tile and 

    im_coll_pre_tile_asc = ee.ImageCollection(pre_tile_asc) 
    im_coll_pre_tile_des = ee.ImageCollection(pre_tile_des)
    im_coll_post_tile_asc = ee.ImageCollection(post_tile_asc) 
    im_coll_post_tile_des = ee.ImageCollection(post_tile_des) 


    # add masks.
    pre_tile_asc_med = im_coll_pre_tile_asc.median()
    pre_tile_des_med =  im_coll_pre_tile_des.median()
    post_tile_asc_med = im_coll_post_tile_asc.median()
    post_tile_des_med = im_coll_post_tile_des.median()
    


    if (len(pre_tile_asc) != 0 and len(pre_tile_des) != 0 and len(post_tile_asc) != 0 and len(post_tile_des) != 0):
      diff_des = pre_tile_des_med.subtract(post_tile_des_med)
      diff_asc = pre_tile_asc_med.subtract(post_tile_asc_med)
      sum_des_asc = (diff_asc.add(diff_des)).divide(2)
      # sum_out[0].append(i+1)
      # sum_out[1].append(sum_des_asc)
      exportOneImage_LS(index,sum_des_asc,i+1, folder_name_x)

      #maybe just immediately export this image... with proper tile..
  


### Export one image

In [ ]:
def exportOneImage_LS(index,image, tile_nr, folder_name):
  """"
  This function exports one image which could potentially cover landslide scars, since it covers a pre and post event time.
  The input variable 'index' refers to the index of the landslide database, which region/database of the four databases is selected.
  These four databased are summed in the dbLS. 
  [0] refers to Hiroshima, Japan
  [1] refers to Tsangpo Gorge, China
  [2] refers to Lombok, Indonesia
  [3] refers to Hokkaido, Japan

  The input variable "image" refers to the image to be exportedn, which is of the collection of Copernicus Sentinel-1 imagery from Google Earth Engine, 
  the version which is in decibel.

  The 'region' variable depicts the region which the image represents, here this is bound to a certain tile. 
  Which tile this is, from the amount of tiles available for each region, is defined by the input variable 'tile_nr'.

  The input varialbe 'folder_name' refers to the name of the folder in the Drive to which the exported images
  are to be saved. This folder is made by the function but can also be made beforehand.
  
  """"
  tiles = dbLS[index]["tiles"]
  tiles_amount = tiles.size().getInfo()
  tiles_list = tiles.toList(tiles_amount)
  image_projection = image.projection()
  geom_tile = ee.Feature(tiles_list.get(tile_nr)).geometry().bounds(1,image_projection)
  name_image = "ImageXLS_%s" % (tile_nr) #ImageIMAGENUMBERFROMCOLL_TILENUMBER'
  ee.batch.Export.image.toDrive(
  image = image,
  dimensions = "256x256",          
  region = geom_tile,
  description = name_image,
  fileNamePrefix = name_image, #necessary next to descirptioN?
  folder = folder_name).start()